# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import pandas as pd
import numpy as np

In [6]:
document = ET.parse( './data/mondial_database.xml' )

In [7]:
root = document.getroot()

# Question 1 - Top 10 Countries with the lowest Mortality Rates

In [8]:
root[0][1].text

'1214489'

In [9]:
country = ''
ans = 100
for element in root.iter('country'):
    if element.find('infant_mortality') is None:
        continue
    if float(element.find('infant_mortality').text) < ans:
        country = element.find('name').text
        ans = float(element.find('infant_mortality').text)
print(country, ans)

Monaco 1.81


In [10]:
inf_mor = {}
for element in root.iter('country'):
    country = element.find('name').text
    try:
        mor_rate = float(element.find('infant_mortality').text)
    except AttributeError:
        continue
        
    inf_mor[country] = mor_rate

sorted(inf_mor.items(), key= lambda x:x[1])[:10]

    

[('Monaco', 1.81),
 ('Japan', 2.13),
 ('Norway', 2.48),
 ('Bermuda', 2.48),
 ('Singapore', 2.53),
 ('Sweden', 2.6),
 ('Czech Republic', 2.63),
 ('Hong Kong', 2.73),
 ('Macao', 3.13),
 ('Iceland', 3.15)]

In [11]:
df_infmort = []
for element in root.iter('country'):
    Dict = dict()
    if element.find("infant_mortality") is not None:
        Dict['Country'] = element.find('name').text
        Dict['Inf Mortality Rate'] = float(element.find('infant_mortality').text)
        df_infmort.append(Dict)

df_infmort = pd.DataFrame(df_infmort)
df_infmort.sort_values(by = "Inf Mortality Rate").head(10)


    

,Country,Inf Mortality Rate
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


# Question 2 - Top 10 Cities with the largest Population

In [12]:
temp = []
for element in root.iter('city'):
    name = element.find('name').text
    country = element.get('country')
    if element.find('population') is not None:
    
        for pop in element.findall('population'):
            year = pop.get('year')
            measured = pop.get('measured')
            population = pop.text
            
            temp.append((country, name, year, measured, int(population)))
            
df = pd.DataFrame(temp, columns = ['Country code' ,'City', 'Year', 'Measurement type', 'Population'])
df.Year = pd.DatetimeIndex(df.Year).year


## This construct loops over the city sub element, building a list of all instances of population entries. There are
## some cities (eg Victoria) that appear in a few countries, so have included Country code as a reference
## Question does not specify in detail the requirements of largest population (based on census data? estimate data?
## most recent entry?) so this allows us to filter as we see best.

## Here we will look at the 10 largest city populations with the constraint that measurement has to be of either
## census or estimate type and the 

df = df.loc[df['Measurement type'].notnull()]
grouped = df.groupby(['Country code','City'], as_index=False)
g = grouped.apply(lambda x: x.nlargest(1, columns = 'Year'))
g.set_index('City', inplace= True)


In [13]:
top_10 = g.sort_values('Population', ascending= False)[:10]
top_10.Population = top_10.Population.map('{:,}'.format)
top_10

,Country code,Year,Measurement type,Population
City,,,,
Shanghai,CN,2010,census,"22,315,474"
Istanbul,TR,2012,admin.,"13,710,512"
Mumbai,IND,2011,census,"12,442,373"
Moskva,R,2013,estimate,"11,979,529"
Beijing,CN,2010,census,"11,716,620"
São Paulo,BR,2010,census,"11,152,344"
Tianjin,CN,2010,census,"11,090,314"
Guangzhou,CN,2010,census,"11,071,424"
Delhi,IND,2011,census,"11,034,555"


In [14]:
## Alternative method

In [15]:
city_pop = []
for element in root.iter('city'):
    if element.find('population') is not None:
        Dict = dict()
        Dict['City'] = element.find('name').text
        Dict['Population'] = int(element.findall('population')[-1].text)
        Dict['Measurement Type'] = element.findall('population')[-1].get('measured')
        Dict['Year'] = element.findall('population')[-1].get('year')
        city_pop.append(Dict)
city_pop = pd.DataFrame(city_pop)
city_pop = city_pop.sort_values(by = "Population", ascending = False)
city_pop.Population = city_pop.Population.map('{:,}'.format)
city_pop.head(10)



,City,Measurement Type,Population,Year
1251,Shanghai,census,"22,315,474",2010
707,Istanbul,admin.,"13,710,512",2012
1421,Mumbai,census,"12,442,373",2011
443,Moskva,estimate,"11,979,529",2013
1250,Beijing,census,"11,716,620",2010
2594,São Paulo,census,"11,152,344",2010
1252,Tianjin,census,"11,090,314",2010
974,Guangzhou,census,"11,071,424",2010
1467,Delhi,census,"11,034,555",2011
977,Shenzhen,census,"10,358,381",2010


# Question 3 - 10 ethnic groups with the largest overall populations

In [16]:
ethnic_pops = []
for element in root.iter('country'):
    country = element.find('name').text
    population = element.findall('population')[-1].text
    for ethnic_group in element.findall('ethnicgroup'):
        Dic = dict()
        Dic['ethnic_group'] = ethnic_group.text
        Dic['pop_percent'] = float(ethnic_group.get('percentage'))
        Dic['country'] = country
        Dic['population'] = int(population)
        ethnic_pops.append(Dic)
    
df_ethnic = pd.DataFrame(ethnic_pops)
df_ethnic['raw_ethnic'] = df_ethnic.population * df_ethnic.pop_percent
g = df_ethnic.groupby('ethnic_group').agg(lambda x:np.dot(x['pop_percent'], x['population']))[['raw_ethnic']].sort_values('raw_ethnic',ascending = False)[:10]
g.raw_ethnic = g.raw_ethnic.map('{:,.0f}'.format)
g
        

,raw_ethnic
ethnic_group,
Han Chinese,"124,505,880,000"
Indo-Aryan,"87,181,558,344"
European,"49,487,221,972"
African,"31,832,512,037"
Dravidian,"30,271,374,425"
Mestizo,"15,773,435,494"
Bengali,"14,677,691,672"
Russian,"13,185,699,608"
Japanese,"12,653,421,200"


In [17]:
### Alternative method

In [18]:
ethnic_pop = []
for element in root.iter('country'):
    country = element.find("name").text
    population = element.findall("population")[-1].text
    for subelement in element.iter("ethnicgroup"):
        Dict = dict()
        Dict['ethnic_group'] = subelement.text
        Dict['pop_percent'] = float(subelement.get("percentage"))
        Dict['country'] = country
        Dict['population'] = int(population)
        ethnic_pop.append(Dict)
ethnic_pop = pd.DataFrame(ethnic_pop)       
ethnic_pop['raw_ethnic'] = ethnic_pop.pop_percent * ethnic_pop.population/100
ethnic_pop.pivot_table(values = "raw_ethnic", index = ["ethnic_group"], aggfunc= np.sum).sort_values(ascending = False).head(10)

ethnic_group
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: raw_ethnic, dtype: float64

# Question 4 - name and country of a) longest river, b) largest lake and c) airport at highest elevation

## a) Name and Country of longest river

In [19]:
country_map = {country.get('car_code'): country.find('name').text for country in root.iter('country')}

In [20]:
rivers = [(river.find('name').text,river.find('source').get('country'), float(river.find('length').text)) 
 for river in root.iter('river') if river.find('length') is not None]
river_df = pd.DataFrame(rivers, columns = ['Name', 'Source', 'Length']).sort_values('Length', ascending = False)[:1]
river_df.Source = river_df.Source.map(country_map)
river_df

,Name,Source,Length
174,Amazonas,Peru,6448.0


In [21]:
rivers = []
for element in root.iter("river"):
    if element.find("length") is not None:
        Dict = dict()
        Dict['river'] = element.find("name").text
        Dict['length'] = float(element.find("length").text)
        Dict['Source_country'] = (element.find("source").get("country")) ## Some have multiple countries
        rivers.append(Dict)
rivers = pd.DataFrame(rivers)
rivers

country_code = dict()
for element in root.iter("country"):
    country_code.update({element.get("car_code"):element.find("name").text})

rivers["Country Name"] = rivers.Source_country.map(country_code)
rivers.sort_values(by = "length", ascending= False).head()

,Source_country,length,river,Country Name
174,PE,6448.0,Amazonas,Peru
137,CN,6380.0,Jangtse,China
136,CN,4845.0,Hwangho,China
123,R,4400.0,Lena,Russia
201,ZRE,4374.0,Zaire,Zaire


## b) Name and Country of Largest Lake

In [64]:
import re

In [95]:
def func(m):
    return country_map[m.group(1)]


In [103]:
lakes = [(lake.find('name').text,lake.get('country'), float(lake.find('area').text), float(lake.find('depth').text)) 
 for lake in root.iter('lake') if lake.find('area') is not None and lake.find('depth') is not None]
lake_df = pd.DataFrame(lakes, columns = ['Name', 'Countries', 'Area', 'Depth']).sort_values('Area', ascending = False)
temp = lake_df.Countries
lake_df.Countries = temp.apply(lambda x: re.sub('(\w+)',func, x))
lake_df[:1]

,Name,Countries,Area,Depth
50,Caspian Sea,Russia Azerbaijan Kazakhstan Iran Turkmenistan,386400.0,995.0


In [104]:
lakes = []
for element in root.iter("lake"):
    if (element.find("area") is not None) & (element.get("country") is not None):
        Dict = dict()
        Dict['lake'] = element.find("name").text
        if element.find("depth") is not None:
            Dict['depth'] = float(element.find("depth").text)
        else:
            np.nan
        Dict['area'] = float(element.find("area").text)
        Dict['Source_country'] = (element.get("country")) ## Some have multiple countries
        lakes.append(Dict)
lakes = pd.DataFrame(lakes)
lakes

lakes_first = lakes.Source_country.str.split(' ')
lakes_first = pd.Series([item[0] for item in lakes_first])
lakes["Country Name"] = lakes_first.map(country_code)
lakes.sort_values(by = ["area", "depth"], ascending= False).head(5)


,Source_country,area,depth,lake,Country Name
54,R AZ KAZ IR TM,386400.0,995.0,Caspian Sea,Russia
107,CDN USA,82103.0,405.0,Lake Superior,Canada
79,EAT EAK EAU,68870.0,85.0,Lake Victoria,Tanzania
104,CDN USA,59600.0,229.0,Lake Huron,Canada
106,USA,57800.0,281.0,Lake Michigan,United States


## c) Airport at highest elevation

In [164]:
airports = []
for el in root.iter('airport'):
    airport_name = el.find('name').text
    country = el.get("country")
    if el.find('elevation').text is not None:
        airports.append((airport_name, country, el.find('elevation').text))
airports = pd.DataFrame(airports, columns = ['Airport Name', 'Country', 'Elevation']).dropna().set_index('Airport Name')
airports['Country'] = airports.Country.map(country_map)
airports['Elevation'] = airports.Elevation.map(int)
airports.sort_values('Elevation', ascending = False)[:1]
        

,Country,Elevation
Airport Name,,
El Alto Intl,Bolivia,4063


In [165]:
## Alternative

In [166]:
airports = []
for element in root.iter("airport"):
    if element.find("elevation") is not None:
        Dict = dict()
        Dict["Airport"] = (element.find("name").text)
        Dict["Country"] = (element.get("country"))
        Dict["Elevation"] = (element.find("elevation").text)
        airports.append(Dict)
airports = pd.DataFrame(airports).dropna()
airports.Elevation = airports.Elevation.map(int)
airports['Country Name'] = airports.Country.map(country_code)
airports.sort_values(by = "Elevation", ascending = False).head()


,Airport,Country,Elevation,Country Name
80,El Alto Intl,BOL,4063,Bolivia
219,Lhasa-Gonggar,CN,4005,China
241,Yushu Batang,CN,3963,China
813,Juliaca,PE,3827,Peru
815,Teniente Alejandro Velasco Astete Intl,PE,3311,Peru
